<a href="https://colab.research.google.com/github/NikitaSavvin2000/CNN-LSTM/blob/main/new_lstm_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tqdm import tqdm
# Загрузка данных
# Предполагается, что у вас есть DataFrame df_Pl с колонками "time" и "P_l"
# "time" содержит временной ряд в формате 'yyyy-mm-dd hh-mm-ss'
# "P_l" содержит значения энергопотребления

df_Pl = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vRKJzXJlQSNew0dxrQ8mFQMhBv_4owvfsF2If1b-rmxMZkR5gabHC4OiaSwt8Ul1Omc8taR27UohSeg/pub?gid=429486365&single=true&output=csv')
nan_indices = df_Pl.index[df_Pl['P_l'].isna()]
avg_pl = df_Pl['P_l'].mean()

df_Pl['P_l'].fillna(avg_pl, inplace=True)


# Преобразование времени в формат datetime
df_Pl['time'] = pd.to_datetime(df_Pl['time'])

# Нормализация данных
scaler = MinMaxScaler(feature_range=(0, 1))
df_Pl['Scaled_P_l'] = scaler.fit_transform(df_Pl['P_l'].values.reshape(-1, 1))

# Создание временных последовательностей
look_back = 10  # можно изменить в зависимости от требований
X, y = [], []

for i in tqdm(range(len(df_Pl) - look_back)):
    X.append(df_Pl[['Scaled_P_l', 'time']][i:(i + look_back)].values)
    y.append(df_Pl['Scaled_P_l'][i + look_back])

X, y = np.array(X), np.array(y)




100%|██████████| 762881/762881 [46:52<00:00, 271.25it/s]


In [ ]:
from tensorflow.keras.layers import Input, concatenate, LSTM, Dense, Conv1D, MaxPooling1D, Flatten
from tensorflow.keras.models import Model
import numpy as np

# Разделение на обучающий и тестовый наборы
train_size = int(len(X) * 0.9)
test_size = len(X) - train_size
X_train, X_test = X[0:train_size, :], X[train_size:len(X), :]
y_train, y_test = y[0:train_size], y[train_size:len(y)]

# Преобразование данных в формат, подходящий для LSTM и CNN
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 2))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 2))

# Создание входов
input_cnn = Input(shape=(X_train.shape[1], 1), name='input_cnn')
input_lstm = Input(shape=(X_train.shape[1], 1), name='input_lstm')

# Сверточный слой для признака P_l
conv1 = Conv1D(filters=32, kernel_size=3, activation='relu')(input_cnn)
pool1 = MaxPooling1D(pool_size=2)(conv1)
flat1 = Flatten()(pool1)

# Вход для времени
flat2 = Flatten()(input_lstm)

# Объединение выходов сверточного слоя и выходов для времени
merged = concatenate([flat1, flat2])

# Полносвязный слой
dense = Dense(units=50, activation='relu')(merged)

# Выходной слой
output = Dense(units=1, activation='linear')(dense)

# Создание и обучение модели
model = Model(inputs=[input_cnn, input_lstm], outputs=output)
model.compile(optimizer='adam', loss='mean_squared_error')

X_train_cnn = X_train[:, :, 0:1].astype('float32')
X_train_lstm = np.vectorize(lambda x: x.timestamp())(X_train[:, :, 1]).astype('float32')
model.fit([X_train_cnn, X_train_lstm], y_train, epochs=10, batch_size=512)


Epoch 1/10
1341/1341 [==============================] - 8s 6ms/step - loss: 9236686831616.0000
Epoch 2/10
1341/1341 [==============================] - 8s 6ms/step - loss: 741.8521
Epoch 3/10
1341/1341 [==============================] - 8s 6ms/step - loss: 37995075862528.0000
Epoch 4/10
1341/1341 [==============================] - 6s 5ms/step - loss: 599.4373
Epoch 5/10
1341/1341 [==============================] - 8s 6ms/step - loss: 600.2780
Epoch 6/10
1134/1341 [========================>.....] - ETA: 0s - loss: 26343504871424.0000

In [ ]:
# Функция для прогнозирования
def predict_energy_consumption(dates_to_predict):
    # Преобразование входных дат в формат datetime
    dates_to_predict = pd.to_datetime(dates_to_predict)

    # Создание временных последовательностей для прогноза
    X_predict = []

    for date in dates_to_predict:
        # Получение последних look_back точек для каждой даты
        X_predict.append(df_Pl[df_Pl['time'] <= date].tail(look_back)[['Scaled_P_l', 'time']].values)

    X_predict = np.array(X_predict)

    # Преобразование данных в формат, подходящий для LSTM
    X_predict = np.reshape(X_predict, (X_predict.shape[0], X_predict.shape[1], 2))

    # Предсказание энергопотребления
    predicted_scaled_values = model.predict(X_predict)

    # Инвертирование нормализации для получения итоговых прогнозов
    predicted_values = scaler.inverse_transform(predicted_scaled_values)

    # Создание DataFrame с прогнозами
    df_predict = pd.DataFrame({'time': dates_to_predict, 'P_l': predicted_values.flatten()})

    return df_predict

In [ ]:
# Пример использования функции для прогнозирования
import matplotlib.pyplot as plt



dates_to_predict = pd.date_range(start='2023-09-01', end='2023-09-10', freq='5T')
predicted_df = predict_energy_consumption(dates_to_predict)

# Вывод результатов
print(predicted_df)

plt.figure(figsize=(14, 7))
plt.plot(predicted_df['time'], predicted_df['P_l'], linewidth=0.2)
plt.title(label="Dataset with Filled Values using Poisson method. Count gaps = 10%")
plt.ylabel("P_l Value")
plt.xlabel("Time")
plt.show()
